In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

# ne: number of circunscription
# nlg: number of coalition


def get_data(
  election_date = "14/06/1987",
  district_id = 1,
  nlg = 1
):
  api = f"https://elezionistorico.interno.gov.it/candidati.php?tpel=C&dtel={election_date}&tpa=I&tpe=I&lev0=0&levsut0=0&lev1=32&levsut1=1&ne1=32&es0=S&es1=S&ms=S&ne={district_id}&nlg={nlg}&ts=C"
  r = requests.get(api,  headers=headers)

  soup = BeautifulSoup(r.text)
  party = soup.select_one(".simbolo_candidati").text
  df = pd.read_html(r.text, thousands=".")[0]
  df["party"] = party
  df["party_id"] = nlg
  df["district_id"] = district_id

  return df

In [3]:
election_date = "14/06/1987"
output = []
for district_id in range(1, 32 + 1):
  nlg = 1
  while nlg:
    try:
      frag = get_data(
        district_id = district_id,
        nlg = nlg,
        election_date=election_date
      )
      output.append(frag)
      nlg += 1
    except:
      nlg = None
    

In [ ]:
df = pd.concat(output, ignore_index=True)
df = df.drop(columns=["Data di nascita", "Luogo di nascita"])
df = df.rename(columns={"Candidati": "candidate", "Preferenze": "value", "Unnamed: 4": "elected"})
df["value"] = df["value"].replace("-", 0).astype(int)
df.head(5)

,candidate,value,elected,party,party_id,district_id
0,PASTORE GIULIO,74017,Eletto,DC,1,1
1,SCALFARO OSCAR LUIGI,55499,Eletto,DC,1,1
2,PELLA GIUSEPPE,50814,Eletto,DC,1,1
3,RAPELLI GIUSEPPE ANTONIO,36238,Eletto,DC,1,1
4,MENOTTI NATALE,35238,Eletto,DC,1,1


In [ ]:
year = election_date[6:12]

df.to_csv(f"../data_output/Italy/{year}_camera_district.csv.gz", index=False, compression="gzip")